- Dues queries: `"#LaRevuelta"` i `"@LaRevueltaTVE"` per un seguit de franges temporals, fent servir `since:` i `until:`
- Es crea un diccionari de respostes de l'API segons aquestes dates
- Processament de dades

In [ ]:
import asyncio
import pickle
from datetime import datetime, timedelta

import pandas as pd
from twitter_scrape_utils import setup_accounts, tweets2df
from twscrape import API, gather
from twscrape.logger import set_log_level

set_log_level("DEBUG")
api = API()  # initialize the API with default account database `accounts.db`

pd.set_option("display.max_colwidth", None)

In [ ]:
await setup_accounts(api)

- màxim de 3 tweets per dia pel 2023 -> l'API s'ho passa pel forro
- màxim de 2 respostes (que siguin del compte @RAEinforma) 

In [ ]:
from datetime import datetime, timedelta

start_date = datetime(2025, 1, 13)
end_date = datetime(2025, 1, 19)

date_queries = {}
current_date = start_date
while current_date <= end_date:
    since_date = current_date.strftime("%Y-%m-%d")
    until_date = (current_date + timedelta(days=1)).strftime("%Y-%m-%d")
    # query = f"#LaRevuelta since:{since_date} until:{until_date}"
    query = f"@LaRevuelta_TVE since:{since_date} until:{until_date}"
    date_queries[since_date] = query

    current_date += timedelta(days=1)

In [ ]:
date_queries

In [ ]:
tweets = {}
# Load already scrapped tweets
# with open("tweets#.pkl", "rb") as f:
#     tweets = pickle.load(f)
# with open("tweets@.pkl", "rb") as f:
#     tweets2 = pickle.load(f)

In [ ]:
for date, query in date_queries.items():
    print(date)
    if date in tweets:
        continue
    tweets[date] = await gather(api.search(query, limit=1000))

In [ ]:
for key in tweets.keys():
    print(key, len(tweets[key]))

In [ ]:
# Save tweets
# with open("tweets.pkl", "wb") as f:
#     pickle.dump(tweets, f)

In [ ]:
def parse_tweet(tweet):
    return {
        "id": tweet.id,
        "url": tweet.url,
        "username": tweet.user.username,
        "content": tweet.rawContent,
        "created_at": tweet.date,
        "likes": tweet.likeCount,
        "retweets": tweet.retweetCount,
        "replies": tweet.replyCount,
        "parent": tweet.inReplyToTweetId,
        "photos": tweet.media.photos,
        "links": tweet.links,
        "location": tweet.user.location,
    }

In [ ]:
def get_df(tweets):
    dfs = {k: tweets2df(v, pase_tweet) for k, v in tweets.items()}

    df = pd.concat(list(dfs.values()))
    df["dia"] = df.created_at.astype(str).str[:10]
    # df["mes"] = df.created_at.astype(str).str[:7]
    df["any"] = df.created_at.astype(str).str[:4]
    df["hora"] = df.created_at.astype(str).str[11:19]
    df.location = df.location.str.lower().str.strip()

    df = df.query("any == '2025'").drop_duplicates(subset="id")
    return df

In [ ]:
MAX_ROWS = 100


def merge_dfs(tweets, tweets2, key):
    df = tweets2df(tweets[key])
    df2 = tweets2df(tweets2[key])
    df["origin"] = "#LaRevuelta"
    df2["origin"] = "@LaRevuelta_TVE"
    df = pd.concat([df, df2])
    df.loc[df.duplicated(subset="id", keep=False), "origin"] = "both"
    df["dia"] = df.created_at.astype(str).str[:10]
    # df["mes"] = df.created_at.astype(str).str[:7]
    df["any"] = df.created_at.astype(str).str[:4]
    df["hora"] = df.created_at.astype(str).str[11:19]
    df = (
        df[df.content.str.contains("#LaRevuelta") | df.content.str.contains("@LaRevuelta_TVE")]
        .drop_duplicates(subset="id")
        .query("any == '2025'")
        .sort_values(by=["replies", "retweets", "likes"], ascending=False)  # .iloc[:MAX_ROWS]
        .sort_values(by=["hora"])
        .drop(columns=["created_at", "parent", "any", "origin"])
    )
    return df

In [ ]:
dfs = {k: merge_dfs(tweets, tweets2, k) for k in date_queries.keys()}

In [ ]:
for k, df in dfs.items():
    print(k, ":", len(df))
    df.to_excel(f"{k}.xlsx", index=False)